# Send text to Information Discovery and highlight results as HTML

In this tutorial we want to show you how you can send text to the Information Discovery web service and output the result as highlighted HTML. We start with the method that converts the Json output to HTML.

### Convert Json to HTML

The following method converts the Json result to HTML code. The offsets (begin and end) of an annotation are recalculated after each annotation. Multiple annotations on a text passage are supported in principle, whereby only one of them is randomly highlighted. Overlapping annotations are not supported in this approach.

In [3]:
from collections import defaultdict
from IPython.core.display import display, HTML
import os
import requests
import json
from operator import itemgetter

def json2html(jData):
    #First extract the document text from the return object. The offsets should be calculated on this object. 
    for annotation in jData['annotationDtos']:  
        if (annotation['type'] == 'de.averbis.types.health.DocumentAnnotation'):
            document = annotation['coveredText']
    
    offset = 0
    offset_dict= {}
    lastEnd = 0
    lastOffset = 0
    for annotation in sorted(jData['annotationDtos'], key=itemgetter('begin', 'end')):
        if (annotation['type'] in annotations.keys()):
            color = annotations[annotation['type']]
            tooltip = ''
            if 'conceptId' in annotation and 'dictCanon' in annotation:
                tooltip = annotation['conceptId'] + " " + annotation['dictCanon']
            spanBegin = '<span title="' + tooltip + '" style="text-decoration: underline; text-decoration-color: ' + color + ';">'
            spanEnd = '</span>'
            # negated concepts shall be striked through in the HTML output
            if ('negatedBy' in annotation and annotation['negatedBy'] is not None):
                spanBegin = spanBegin + '<s>'
                spanEnd = '</s>' + spanEnd
            begin = annotation['begin'] + offset
            #If the current annotation is before the end of the last annotation, the begin must be adjusted.
            if (annotation['begin'] < lastEnd):
                begin = begin - lastEndOffset
            end = annotation['end'] + offset
            offset = offset + len(spanBegin) + len(spanEnd)
            lastEnd = annotation['end']
            lastEndOffset = len(spanEnd)
            document = document[:begin] + spanBegin + document[begin:end] + spanEnd + document[end:]  
    return document



### Define the parameters and run the application

Now you can set some helpful parameters.

 - text: here you enter the text you want to analyze.
 - project: this is the name of the Information Discovery project
 - pipeline: The name of the pipeline in Information Discovery
 - language: the language of the text
 - url: the URL of the Information Discovery. In my example I send it to the Health Discovery demo server
 - annotations: The annotations dictionary defines as key the type your want to get highlighted, as value the color in the highlighting should be done. In our example we want to highlight diagnoses, medications and lab values. 
 
An overview of all types that Information Discovery can extract can be found in the documentation at https://averbis.com/support



In [8]:
txt= "The patient had a cold and a headache, but no fever. He was on Aspirin."

project = "default"
pipeline = "discharge"
language = "en"
url = 'https://health-discovery.averbis.com/demo/rest/textanalysis/projects/' + project + '/pipelines/' + pipeline + '/analyseText?language=' + language
annotations = {'de.averbis.types.health.Diagnosis' : 'orange', 
               'de.averbis.types.health.Medication': 'limegreen',
               'de.averbis.types.health.LaboratoryValue': 'LightSkyBlue'}

#All we have to do now is send the text to Information Discovery. Then we convert the Json to HTML and output it here.
headers = {'Content-type': 'text/plain; charset=utf-8'}
myResponse = requests.post(url, data=txt.encode('utf-8'), headers=headers)
if (myResponse.ok):
    jData = json.loads(myResponse.content)
    display(HTML('<h2>' + json2html(jData) + '</h2>'))